# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [21]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

# Loading
A_df = pd.read_csv("model-A.tsv", sep="\t")
B_df = pd.read_csv("model-B.tsv", sep="\t")
Q_df = pd.read_csv("cost-Q.tsv",  sep="\t")
R_df = pd.read_csv("cost-R.tsv",  sep="\t")

state_cols = ["x", "v", "a"] if set(["x","v","a"]).issubset(A_df.columns) else A_df.columns.tolist()
A = A_df[state_cols].to_numpy(float)
Q = Q_df[state_cols].loc[:, state_cols].to_numpy(float)

act_col = "u" if "u" in B_df.columns else B_df.columns[0]
B = B_df[[act_col]].to_numpy(float)
R = (R_df[[act_col]] if act_col in R_df.columns else R_df.iloc[:, :1]).to_numpy(float)

def solve_dare(A, B, Q, R, tol=1e-12, max_iter=10000):
    P = Q.copy()
    AT, BT = A.T, B.T
    for _ in range(max_iter):
        S  = R + BT @ P @ B                
        K  = np.linalg.solve(S, BT @ P @ A) 
        Pn = Q + AT @ P @ A - AT @ P @ B @ K
        if np.linalg.norm(Pn - P, ord='fro') < tol:
            P = Pn
            break
        P = Pn

    S = R + BT @ P @ B
    K = np.linalg.solve(S, BT @ P @ A)
    return K, P

K, _ = solve_dare(A, B, Q, R)   


Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [22]:
# YOUR CHANGES HERE

out = pd.DataFrame([K.flatten()], columns=state_cols)
out.to_csv("control-K-intended.tsv", sep="\t", index=False, float_format="%.9f")
print("Wrote control-K-intended.tsv")
out

Wrote control-K-intended.tsv


,x,v,a
0,0.33446,1.304454,1.858131


Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [23]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

# Loading
K = pd.read_csv("control-K-intended.tsv", sep="\t").to_numpy(float).reshape(1, -1)  
log = pd.read_csv("qc-log.tsv", sep="\t")

state_cols = ["x", "v", "a"]
missing = [c for c in state_cols if c not in log.columns]
if missing:
    raise ValueError(f"qc-log.tsv missing columns: {missing}")

X = log[state_cols].to_numpy(float)         
u_check = (-K @ X.T).flatten()              

if "index" in log.columns:
    idx = log["index"].to_numpy()
else:
    idx = np.arange(len(log))



Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [24]:
# YOUR CHANGES HERE

qc_check = pd.DataFrame({"index": idx, "u_check": u_check})
qc_check.to_csv("qc-check.tsv", sep="\t", index=False, float_format="%.9f")
print(f"Wrote qc-check.tsv with {len(qc_check)} rows.")
qc_check.head()

Wrote qc-check.tsv with 100 rows.


,index,u_check
0,0,1.672299
1,1,-1.152095
2,2,-1.235183
3,3,-0.288504
4,4,0.369202


Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [25]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

# Loading
log = pd.read_csv("qc-log.tsv", sep="\t")

state_cols = ["x", "v", "a"]
missing = [c for c in state_cols if c not in log.columns]
if missing:
    raise ValueError(f"qc-log.tsv missing columns: {missing}")

X = log[state_cols].to_numpy(float)                
u_cols = [c for c in log.columns if c not in state_cols + ["index"]]
if not u_cols:
    raise ValueError("Could not find any action column in qc-log.tsv.")
u = log[u_cols[0]].to_numpy(float)                 

w, *_ = np.linalg.lstsq(X, u, rcond=None)          
K_actual = -w.reshape(1, -1)                       

Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [26]:
# YOUR CHANGES HERE
K_df = pd.DataFrame(K_actual, columns=state_cols)
K_df.to_csv("control-K-actual.tsv", sep="\t", index=False, float_format="%.9f")
print("Wrote control-K-actual.tsv")
K_df


Wrote control-K-actual.tsv


,x,v,a
0,0.340438,1.30012,1.950117


Submit "control-k-actual.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [27]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd

train = pd.read_csv("qc-train.tsv", sep="\t").copy()

state_cols = ["x", "v", "a"]
u_col = "u" if "u" in train.columns else [c for c in train.columns if c not in state_cols + ["index"]][0]

if "index" in train.columns:
    train = train.sort_values("index").reset_index(drop=True)

X_t   = train[state_cols].to_numpy(float)            
U_t   = train[[u_col]].to_numpy(float)               
X_tp1 = train[state_cols].shift(-1).to_numpy(float)  

X_t   = X_t[:-1, :]
U_t   = U_t[:-1, :]
X_tp1 = X_tp1[:-1, :]

Phi = np.hstack([X_t, U_t])                          
W, *_ = np.linalg.lstsq(Phi, X_tp1, rcond=None)      

A_new = W[:3, :].T                                   
B_new = W[3:, :].T                                   




Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [28]:
# YOUR CHANGES HERE

A_df = pd.DataFrame(A_new, columns=state_cols)
B_df = pd.DataFrame(B_new, columns=[u_col])

A_df.to_csv("model-A-new.tsv", sep="\t", index=False, float_format="%.9f")
B_df.to_csv("model-B-new.tsv", sep="\t", index=False, float_format="%.9f")

print("Saved model-A-new.tsv and model-B-new.tsv")
print("A_new:\n", A_df)
print("\nB_new:\n", B_df)


Saved model-A-new.tsv and model-B-new.tsv
A_new:
               x             v             a
0  1.000000e+00  1.100000e+00  2.886580e-15
1 -1.281955e-16  9.000000e-01  9.500000e-01
2 -9.313780e-17  5.551115e-16  1.100000e+00

B_new:
               u
0 -1.110223e-16
1 -1.000000e-02
2  9.000000e-01


Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [29]:
text = """
Acknowledgments

People I discussed the assignment with:
none

Extra libraries used (beyond the course content):
none

Use of generative AI:
none
"""

with open("acknowledgments.txt", "w", encoding="utf-8") as f:
    f.write(text)

print("Saved acknowledgments.txt")


Saved acknowledgments.txt
